### spaCy installieren; Erforderliche Pakete importieren

In [2]:
# spacy installieren und das deutsche Sprachmodell herunterladen
!pip install spacy
!python -m spacy download de_core_news_sm # kleines Modell
!python -m spacy download de_core_news_md # mittel-großes Modell

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-11 21:51:13.406587: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-11 21:51:14.360943: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-11 21:51:14.361065: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could no

### Eine Verarbeitungspipeline erstellen; auf einen Text anwenden

Hinweis: spaCy-Pipelines sind in hohem Maße anpassbar und erlauben sogar ein erneutes Training der Komponenten. Dies würde den Rahmen dieses Einsteigerkurses sprengen, ebenso wie die Funktion bestimmter Komponenten. Daher hier ein kurzes Glossar für die Standardkomponenten; Details finden Sie in der Dokumentation. 

- tok2vec: konvertiert Token in Vektoren, die als Eingabe an spätere Modelle übergeben werden
- tagger: POS-Tagger
- morphologizer: sagt morphologische Merkmale und grobkörnige POS-Tags für Token voraus
- parser: übergangsbasierter syntaktischer Abhängigkeits-Parser, der syntaktische Abhängigkeiten zwischen Token identifiziert
- ner: Named Entity Recognizer
- attribute_ruler: erlaubt das Erstellen von Token-Mustern und das Setzen bestimmter Attribute für Token, die durch diese Muster identifiziert werden
- lemmatizer: Lemmatisierer

In [3]:
import spacy
import de_core_news_sm
#import de_core_news_md
import pandas as pd

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
# nlp = de_core_news_md.load()
nlp = de_core_news_sm.load()
print(f'Pipeline components: {nlp.pipe_names}')

Pipeline components: ['tok2vec', 'tagger', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']


In [5]:
text = "Berlin ist die Hauptstadt von Deutschland und Sitz des Deutschen Bundestages. " \
       "Es ist ein Zentrum der Kultur und Innovation und ein Muss für jeden Touristen, der das Land besucht."
doc = nlp(text)

In [6]:
# Der Originaltext kann immer leicht eingesehen werden
doc.text

'Berlin ist die Hauptstadt von Deutschland und Sitz des Deutschen Bundestages. Es ist ein Zentrum der Kultur und Innovation und ein Muss für jeden Touristen, der das Land besucht.'

### Sätze

In [7]:
sents = list(doc.sents)
sents

[Berlin ist die Hauptstadt von Deutschland und Sitz des Deutschen Bundestages.,
 Es ist ein Zentrum der Kultur und Innovation und ein Muss für jeden Touristen, der das Land besucht.]

### Tokens

In [8]:
# Token: die Wörter, wie sie ursprünglich im Dokument erscheinen 
tokens = [token for token in doc]
token_texts = [token.text for token in tokens]
' | '.join(token_texts)

'Berlin | ist | die | Hauptstadt | von | Deutschland | und | Sitz | des | Deutschen | Bundestages | . | Es | ist | ein | Zentrum | der | Kultur | und | Innovation | und | ein | Muss | für | jeden | Touristen | , | der | das | Land | besucht | .'

### Lemmas

In [9]:
lemmas = [token.lemma_ for token in doc]
' | '.join(lemmas)

'Berlin | sein | der | Hauptstadt | von | Deutschland | und | Sitz | der | deutsch | Bundestag | -- | es | sein | ein | Zentrum | der | Kultur | und | Innovation | und | ein | Muss | für | jeder | Tourist | -- | der | der | Land | besuchen | --'

### Part-of-Speech Tags

In [10]:
pos_tags = [token.pos_ for token in doc]
' | '.join(pos_tags)

'PROPN | AUX | DET | NOUN | ADP | PROPN | CCONJ | NOUN | DET | ADJ | NOUN | PUNCT | PRON | AUX | DET | NOUN | DET | NOUN | CCONJ | NOUN | CCONJ | DET | NOUN | ADP | DET | NOUN | PUNCT | PRON | DET | NOUN | VERB | PUNCT'

### Noun Chunks

In [11]:
noun_chunk_texts = [str(c) for c in doc.noun_chunks]
' | '.join(noun_chunk_texts)

'Berlin | die Hauptstadt | Deutschland | Sitz | des Deutschen Bundestages | Es | ein Zentrum | der Kultur | Innovation | ein Muss | jeden Touristen | der | das Land'

In [12]:
chunk_labels = []
chunk_word_idxs = set(word.idx for chunk in doc.noun_chunks for word in chunk)
for word in doc:
    
    # Word is not part of a chunk
    if word.idx not in chunk_word_idxs:
        chunk_labels.append('O')
    
    # Word is part of a chunk
    else:
        if word.idx == 0 or word.nbor(-1).idx not in chunk_word_idxs:
            chunk_labels.append('B-CHUNK')
        else:
            chunk_labels.append('I-CHUNK')

for word, cl in zip([w for w in doc], chunk_labels):
    print(f"{word}: {cl}")

Berlin: B-CHUNK
ist: O
die: B-CHUNK
Hauptstadt: I-CHUNK
von: O
Deutschland: B-CHUNK
und: O
Sitz: B-CHUNK
des: I-CHUNK
Deutschen: I-CHUNK
Bundestages: I-CHUNK
.: O
Es: B-CHUNK
ist: O
ein: B-CHUNK
Zentrum: I-CHUNK
der: I-CHUNK
Kultur: I-CHUNK
und: O
Innovation: B-CHUNK
und: O
ein: B-CHUNK
Muss: I-CHUNK
für: O
jeden: B-CHUNK
Touristen: I-CHUNK
,: O
der: B-CHUNK
das: I-CHUNK
Land: I-CHUNK
besucht: O
.: O


### Named Entities

In [13]:
entities = list(doc.ents)
entities

[Berlin, Deutschland, Deutschen Bundestages.]

In [14]:
entity_labels = [f"{token.ent_iob_}-{token.ent_type_}" if token.ent_iob_ != "O" else "O" for token in tokens]
entity_labels

['B-LOC',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

### Den bearbeiteten Text anzeigen

In [15]:
processed_df = pd.DataFrame({"Token": tokens,
                             "Lemma": lemmas,
                             "POS Tag ": pos_tags,
                             "Noun Chunks": chunk_labels,
                             "Labelled Entities": entity_labels
                     })
display(processed_df.head(50))

,Token,Lemma,POS Tag,Noun Chunks,Labelled Entities
0,Berlin,Berlin,PROPN,B-CHUNK,B-LOC
1,ist,sein,AUX,O,O
2,die,der,DET,B-CHUNK,O
3,Hauptstadt,Hauptstadt,NOUN,I-CHUNK,O
4,von,von,ADP,O,O
5,Deutschland,Deutschland,PROPN,B-CHUNK,B-LOC
6,und,und,CCONJ,O,O
7,Sitz,Sitz,NOUN,B-CHUNK,O
8,des,der,DET,I-CHUNK,O
9,Deutschen,deutsch,ADJ,I-CHUNK,B-ORG
